In [1]:
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

In [2]:
SEED = 123
bar_format = '{desc}{percentage:5.1f}%|{bar:10}{r_bar}'

FILENAME = (
    "{dataset_name}"
    "-refine_ts={dataset__refine_ts}"
    "-downsample"
    "-vs={orth_planes_extraction__downsample__voxel_size}"
    "-upsample={orth_planes_extraction__downsample__upsample}"
)

In [3]:
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], ".."))

In [4]:
import os
from easydict import EasyDict
import pickle

from tqdm import tqdm

from registration_metrics import (
    seed_everything,
    CARLA_dataset,
    SemanticKITTI_dataset,
    prepare_data,
    sampling_pipeline,
    load_config,
    get_filename_fmt,
    new_value_if_none,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
from functools import partial


def run_experiment(
    seed, config, filename,
    *,
    override=False,
    map_size=10, N_samples=5, cov_scaler=10,
    # start_internal_id=0, end_internal_id=2422, step_internal_id=40,
    start_internal_id=None, end_internal_id=None, step_internal_id=None,
    prepare_func=prepare_data, dataset=None,  # "CARLA",
    verbose=False
):
    # Load config
    if isinstance(config, str):
        config = load_config(config)

    # Prepare dataset_builder
    dataset_builder_kwargs = {}
    dataset = dataset or config.dataset.name
    if isinstance(dataset, str):
        dataset = dataset.lower()
        assert dataset in ("carla", "semantickitti")
        if dataset == "carla":
            dataset_builder = CARLA_dataset
            dataset_builder_kwargs["dataset_coords_type"] = "world"
        elif dataset == "semantickitti":
            dataset_builder = SemanticKITTI_dataset
    else:
        dataset_builder = dataset

    # Set begin, end and step for external ids for dataset
    if dataset_builder is CARLA_dataset:
        start_internal_id = new_value_if_none(start_internal_id, 0)
        end_internal_id = new_value_if_none(end_internal_id, 2422)
        step_internal_id = new_value_if_none(step_internal_id, 40)
    elif dataset_builder is SemanticKITTI_dataset:
        start_internal_id = new_value_if_none(start_internal_id, 0)
        end_internal_id = new_value_if_none(end_internal_id, 3630)
        step_internal_id = new_value_if_none(step_internal_id, 120)
    else:
        raise ValueError('Unknown dataset_builder type.')

    # Determine dataset name
    dataset_name = dataset_builder.__name__
    dataset_name_suffix = "_dataset"
    if dataset_name.endswith(dataset_name_suffix):
        dataset_name = dataset_name[:-len(dataset_name_suffix)]

    # Determine filename for results
    filename_fmt = get_filename_fmt(config)
    filename_fmt["dataset_name"] = dataset_name
    filename = filename.format(**filename_fmt) + ".pkl"
    print(f"Results WILL be saved to \"{filename}\".")

    # Check existance of file for results
    if os.path.exists(filename):
        print(f"File \"{filename}\" for results exists! ", end="")
        if override:
            print("Continuing.")
        else:
            print("Exiting.")
            return

    seed_everything(seed)

    results = {}

    try:
        # Main loop
        for internal_id in tqdm(
            range(start_internal_id, end_internal_id, step_internal_id),
            desc="Internal IDs",
            bar_format=bar_format
        ):
            start_id, end_id = internal_id, internal_id + map_size

            data = dataset_builder(start_id=start_id, end_id=end_id,
                                   **dataset_builder_kwargs)

            pcs = data.clds
            T_gt = data.poses

            pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
                pcs, T_gt, config)
            details["orth_planes"] = orth_planes
            details["orth_planes_cld_ind"] = orth_planes_cld_ind

            tips = EasyDict(orth_list=orth_planes, config=config)

            save_dict = sampling_pipeline(
                pcs=pcs, T_gt=T_gt, map_tips=tips,
                N_samples=N_samples, cov_scaler=cov_scaler,
                orth_planes_cld_ind=orth_planes_cld_ind,
                verbose=verbose
            )

            results[internal_id, map_size] = {**details, **save_dict}
            
    except KeyboardInterrupt:
        print("Process was interrupted! Probably something is missed!")

    with open(filename, 'wb') as sfile:
        pickle.dump(results, sfile)
    print(f"Results were saved to \"{filename}\".")


run_SemanticKITTI_seq0 = partial(run_experiment, dataset="semantickitti")
run_CARLA_seq5 = partial(run_experiment, dataset="carla")

# SemanticKITTI

## SVD planes extractor + refined transformations

In [22]:
filename = FILENAME
run_SemanticKITTI_seq0(SEED, "downsampling_vs0.4-refine_ts", filename)

{'dataset': {'refine_ts': True},
 'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 0.4},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Results WILL be saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False.pkl".
File "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False.pkl" for results exists! Continuing.
Random seed set as 123


Internal IDs100.0%|██████████| 31/31 [26:38<00:00, 51.57s/it]

Results were saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False.pkl".


С оптимизированной метрикой МоМ посчиталось за 26 минут 38 секунд!

## SVD planes extractor + MOM on planes + refined transformations

In [8]:
filename = FILENAME + "-mom_{metrics__mom__input}"
run_SemanticKITTI_seq0(SEED, "downsampling_vs0.4-refine_ts-mom_planes", filename)

{'dataset': {'name': 'semantickitti', 'refine_ts': True},
 'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'planes', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 0.4},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Results WILL be saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False-mom_planes.pkl".
Random seed set as 123


Internal IDs  3.2%|▎         | 1/31 [04:16<2:08:12, 256.42s/it]


Process was interrupted! Probably something is missed!
Results were saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False-mom_planes.pkl".


takes too much time

## SPVCNN planes extractor + refined transformations

In [5]:
filename = FILENAME + "-{planes_segmentation__algo}"
run_experiment(SEED, "spvcnn-downsampling_vs0.4-refine_ts", filename)

{'dataset': {'name': 'semantickitti', 'refine_ts': True},
 'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 0.4},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'spvcnn', 'spvcnn': {'voxel_size': 0.05}}}
Results WILL be saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False-spvcnn.pkl".
Random seed set as 123


Internal IDs100.0%|██████████| 31/31 [15:37<00:00, 30.25s/it]

Results were saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False-spvcnn.pkl".


С оптимизированной метрикой МоМ посчиталось за 15 минут 37 секунд!

## SPVCNN planes extractor + MoM on planes + refined transformations

In [6]:
filename = FILENAME + "-{planes_segmentation__algo}" + "-mom_{metrics__mom__input}"
run_SemanticKITTI_seq0(SEED, "spvcnn-downsampling_vs0.4-mom_planes-refine_ts", filename)

{'dataset': {'name': 'semantickitti', 'refine_ts': True},
 'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'planes', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 0.4},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'spvcnn', 'spvcnn': {'voxel_size': 0.05}}}
Results WILL be saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False-spvcnn-mom_planes.pkl".
Random seed set as 123


Internal IDs100.0%|██████████| 31/31 [11:28<00:00, 22.20s/it]

Results were saved to "SemanticKITTI-refine_ts=True-downsample-vs=0.4-upsample=False-spvcnn-mom_planes.pkl".


С оптимизированной метрикой МоМ посчиталось за 11 минут 28 секунд!